In [10]:
import psycopg2
import pandas as pd
import time
from sqlalchemy import create_engine

In [11]:
DB_NAME = "reviews"
DB_USER = "postgres"
DB_PASS = "ap16"
DB_HOST = "localhost"
DB_PORT = 5432
DB_TABLE = "reviews"

In [12]:
try:
    start_time = time.time()
    conn = psycopg2.connect(database=DB_NAME,
                            user=DB_USER,
                            password=DB_PASS,
                            host=DB_HOST,
                            port=DB_PORT)

    print("Database connected successfully")

    fetch_query = f"SELECT * FROM {DB_NAME}"
    fetch_columns_query = "SELECT column_name FROM information_schema.columns WHERE table_name = 'reviews'"
    
    column_names = []
    cur = conn.cursor()
    cur.execute(fetch_columns_query)

    # Fetch column names from the table for our pandas dataframe
    for s in cur.fetchall():
        column_names.append(s[0])

    # Fetch actual data from postgres and instantiate DataFrame
    cur.execute(fetch_query)
    res = cur.fetchall()
    df = pd.DataFrame(res, columns=column_names)
    
    cur.close()
    conn.close()
    end_time = time.time()
    print(df.head())
    print(f"***** Completed in {(end_time-start_time)} seconds. Dataframe shape : {df.shape}")

except Exception as e:
    print(f"Error : {e.args}")

Database connected successfully
   id helpfulnessnumerator helpfulnessdenominator  \
0   1           B001E4KFG0         A3SGXH7AUHU8GW   
1   2           B00813GRG4         A1D87F6ZCVE5NK   
2   3           B000LQOCH0          ABXLMWJIXXAIN   
3   4           B000UA0QIQ         A395BORC6FGVXV   
4   5           B006K2ZZ7K         A1UQRSCLF8GW1T   

                             score  time  text  productid      userid  \
0                       delmartian     1     1          5  1303862400   
1                           dll pa     0     0          1  1346976000   
2  Natalia Corres "Natalia Corres"     1     1          4  1219017600   
3                             Karl     3     3          2  1307923200   
4    Michael D. Bigham "M. Wassir"     0     0          5  1350777600   

             profilename                                            summary  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1      Not as Advertised  Product arrived labeled as J

In [13]:
df.drop(['profilename'], inplace=True,  axis=1)
df

,id,helpfulnessnumerator,helpfulnessdenominator,score,time,text,productid,userid,summary
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,These are the BEST treats for training and rew...


In [15]:
conn_string = f"postgresql://{DB_NAME}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_TABLE}"
  
db1 = create_engine(conn_string)
conn1 = db1.connect()

df.to_sql('cached_df', con=conn1, if_exists='replace',
          index=False)

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "reviews"

(Background on this error at: https://sqlalche.me/e/14/e3q8)